In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# processing tables
champ_df = pd.read_csv("projectData/champs.csv").sort_values('id')
participants_df = pd.read_csv("projectData/participants.csv")
participants_df['position'] = participants_df['position'].astype('category')
participants_df['role'] = participants_df['role'].astype('category')
participants_df = participants_df.merge(champ_df,left_on='championid',right_on='id',how='left').drop(['id_x', 'id_y'], axis=1)
participants_df.columns.values[7] = 'Champion Name'
participants_df.columns.values[2] = 'Champion ID'
stats_df = pd.read_csv('projectData/stats1.csv')
stats2_df = pd.read_csv('projectData/stats2.csv',low_memory=False)
stats_df = stats_df.append(stats2_df)
stats_df = stats_df.reset_index(drop=True)
participants_df['index'] = participants_df.index
participants_stats_df = participants_df.merge(stats_df,left_on='index',right_on='id')
participants_stats_df = participants_stats_df.drop('index',axis=1)
participants_stats_df = participants_stats_df[1:]

In [3]:
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
participants_stats_df.head(10)

,matchid,player,Champion ID,ss1,ss2,role,position,Champion Name,id,win,...,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood
1,11,1,115,3,4,DUO_SUPPORT,BOT,Ziggs,10,0,...,1,1,0,211,14,1,0,17,3,0
2,11,2,69,4,7,DUO_CARRY,BOT,Cassiopeia,11,0,...,3,1,2,182,14,1,0,13,5,0
3,11,3,157,14,4,SOLO,MID,Yasuo,12,0,...,6,6,0,106,15,0,0,3,0,0
4,11,4,4,4,12,SOLO,TOP,Twisted Fate,13,0,...,4,3,1,159,15,0,0,10,0,0
5,11,5,28,4,11,NONE,JUNGLE,Evelynn,14,1,...,111,81,30,808,16,0,0,8,2,0
6,11,6,51,7,4,DUO_CARRY,BOT,Caitlyn,15,1,...,6,0,6,179,16,1,0,7,0,0
7,11,7,62,14,4,SOLO,TOP,Wukong,16,1,...,6,3,3,48,17,0,0,8,0,0
8,11,8,134,4,1,SOLO,MID,Syndra,17,1,...,7,2,5,440,16,1,0,6,1,1
9,11,9,43,14,4,DUO_SUPPORT,BOT,Karma,18,1,...,2,1,1,260,16,5,0,25,7,0
10,11,10,19,4,11,NONE,JUNGLE,Warwick,19,0,...,2,2,0,236,12,3,0,14,3,0


In [5]:
participants_stats_df.columns

Index(['matchid', 'player', 'Champion ID', 'ss1', 'ss2', 'role', 'position',
       'Champion Name', 'id', 'win', 'item1', 'item2', 'item3', 'item4',
       'item5', 'item6', 'trinket', 'kills', 'deaths', 'assists',
       'largestkillingspree', 'largestmultikill', 'killingsprees',
       'longesttimespentliving', 'doublekills', 'triplekills', 'quadrakills',
       'pentakills', 'legendarykills', 'totdmgdealt', 'magicdmgdealt',
       'physicaldmgdealt', 'truedmgdealt', 'largestcrit', 'totdmgtochamp',
       'magicdmgtochamp', 'physdmgtochamp', 'truedmgtochamp', 'totheal',
       'totunitshealed', 'dmgselfmit', 'dmgtoobj', 'dmgtoturrets',
       'visionscore', 'timecc', 'totdmgtaken', 'magicdmgtaken', 'physdmgtaken',
       'truedmgtaken', 'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', '

In [6]:
cols_to_drop = ['player','matchid','ss1','ss2','id','item1', 'item2', 'item3', 'item4',
       'item5', 'item6', 'trinket', 'kills', 'deaths', 'assists',
       'largestkillingspree', 'largestmultikill', 'killingsprees',
       'longesttimespentliving', 'doublekills', 'triplekills', 'quadrakills',
       'pentakills', 'legendarykills', 'totdmgdealt', 'magicdmgdealt',
       'physicaldmgdealt', 'truedmgdealt', 'largestcrit', 'totdmgtochamp',
       'magicdmgtochamp', 'physdmgtochamp', 'truedmgtochamp', 'totheal',
       'totunitshealed', 'dmgselfmit', 'dmgtoobj', 'dmgtoturrets',
       'visionscore', 'timecc', 'totdmgtaken', 'magicdmgtaken', 'physdmgtaken',
       'truedmgtaken', 'goldearned', 'goldspent', 'turretkills', 'inhibkills',
       'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
       'enemyjunglekills', 'totcctimedealt', 'champlvl', 'pinksbought',
       'wardsbought', 'wardsplaced', 'wardskilled', 'firstblood']
win = participants_stats_df.drop(cols_to_drop,axis=1)

In [7]:
win.head(10)

,Champion ID,role,position,Champion Name,win
1,115,DUO_SUPPORT,BOT,Ziggs,0
2,69,DUO_CARRY,BOT,Cassiopeia,0
3,157,SOLO,MID,Yasuo,0
4,4,SOLO,TOP,Twisted Fate,0
5,28,NONE,JUNGLE,Evelynn,1
6,51,DUO_CARRY,BOT,Caitlyn,1
7,62,SOLO,TOP,Wukong,1
8,134,SOLO,MID,Syndra,1
9,43,DUO_SUPPORT,BOT,Karma,1
10,19,NONE,JUNGLE,Warwick,0


In [33]:
win_position = win.drop(['Champion ID','Champion Name'],axis=1)
win_position.head(10)

,role,position,win
1,DUO_SUPPORT,BOT,0
2,DUO_CARRY,BOT,0
3,SOLO,MID,0
4,SOLO,TOP,0
5,NONE,JUNGLE,1
6,DUO_CARRY,BOT,1
7,SOLO,TOP,1
8,SOLO,MID,1
9,DUO_SUPPORT,BOT,1
10,NONE,JUNGLE,0


In [34]:
win_position = win_position.groupby(['position','role']).count().dropna()

In [35]:
win_position

win
position role                 
BOT      DUO           48698.0
         DUO_CARRY    328477.0
         DUO_SUPPORT  328472.0
         NONE           2700.0
         SOLO           4987.0
JUNGLE   NONE         367049.0
MID      DUO            8919.0
         DUO_CARRY      4547.0
         DUO_SUPPORT    4548.0
         NONE           2479.0
         SOLO         345831.0
TOP      DUO            1236.0
         DUO_CARRY       356.0
         DUO_SUPPORT     356.0
         NONE             54.0
         SOLO         354778.0

In [36]:
win_position['win_rate (%)'] = (win_position['win']/1834520*100).round(3)
win_position

win  win_rate (%)
position role                               
BOT      DUO           48698.0         2.655
         DUO_CARRY    328477.0        17.905
         DUO_SUPPORT  328472.0        17.905
         NONE           2700.0         0.147
         SOLO           4987.0         0.272
JUNGLE   NONE         367049.0        20.008
MID      DUO            8919.0         0.486
         DUO_CARRY      4547.0         0.248
         DUO_SUPPORT    4548.0         0.248
         NONE           2479.0         0.135
         SOLO         345831.0        18.851
TOP      DUO            1236.0         0.067
         DUO_CARRY       356.0         0.019
         DUO_SUPPORT     356.0         0.019
         NONE             54.0         0.003
         SOLO         354778.0        19.339

### Interestingly, do a solo rank as jungler might have the highest win rate compared to other position